<a href="https://colab.research.google.com/github/ITZ-CJ/Cabin-Part-Prediction-Forecasting/blob/main/Cabin_Part_Prediction_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import random
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

In [5]:
# Set the start and end date
start_date = '2015-01-01'
end_date = '2024-01-01'

# Generate date range
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# List of possible defects
defect_list = ['Arm Rest', 'Defect B', 'Defect C', 'Defect D', 'Defect E',
               'Defect F', 'Defect G', 'Defect H', 'Defect I', 'Defect J',
               'Defect K', 'Defect L', 'Defect M', 'Defect N', 'Defect O']

# Create the dataset
data = {
    'date': [],
    'defects': [],
    'quantities': []
}

for date in date_range:
    # Randomly determine the number of defects between 1 and 5
    num_defects = random.randint(1, 5)

    # Choose unique defects
    selected_defects = random.sample(defect_list, num_defects)

    for selected_defect in selected_defects:
        quantity = random.randint(1, 4)

        # Append data to the lists
        data['date'].append(date)
        data['defects'].append(selected_defect)
        data['quantities'].append(quantity)

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('fake_defects_data.csv', index=False)

print(df.head(20))  # Display first 20 rows for better visibility


         date   defects  quantities
0  2015-01-01  Defect N           4
1  2015-01-01  Defect M           4
2  2015-01-02  Defect B           3
3  2015-01-02  Defect C           4
4  2015-01-02  Defect I           2
5  2015-01-03  Defect A           1
6  2015-01-03  Defect D           3
7  2015-01-03  Defect F           2
8  2015-01-03  Defect O           1
9  2015-01-04  Defect G           1
10 2015-01-04  Defect F           2
11 2015-01-05  Defect A           4
12 2015-01-05  Defect E           1
13 2015-01-05  Defect F           1
14 2015-01-05  Defect C           3
15 2015-01-05  Defect J           4
16 2015-01-06  Defect L           3
17 2015-01-06  Defect F           4
18 2015-01-06  Defect C           1
19 2015-01-06  Defect B           4


In [11]:
# Load the dataset
df = pd.read_csv('fake_defects_data.csv', parse_dates=['date'])

# Prepare an empty DataFrame for the forecast report
forecast_report = pd.DataFrame()

# Iterate over each defect
for defect in df['defects'].unique():
    # Filter the data for the current defect
    defect_data = df[df['defects'] == defect]

    # Aggregate to get total quantities per day
    daily_totals = defect_data.groupby('date')['quantities'].sum().reset_index()
    daily_totals.set_index('date', inplace=True)

    # Fit the SARIMA model
    model = SARIMAX(daily_totals['quantities'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    results = model.fit()

    # Forecast the next 365 days
    forecast = results.get_forecast(steps=365)
    forecast_index = pd.date_range(start=daily_totals.index[-1] + pd.Timedelta(days=1), periods=365, freq='D')
    forecast_values = forecast.predicted_mean

    # Prepare the report for this defect
    defect_forecast = pd.DataFrame({
        'date': forecast_index,
        'forecasted_defects': defect,
        'number_of_defects': forecast_values
    })

    # Append to the overall forecast report
    forecast_report = pd.concat([forecast_report, defect_forecast], ignore_index=True)

# Save the forecast report to a CSV file
forecast_report.to_csv('forecasting_report_all_defects.csv', index=False)

# Display the first few rows of the report
print(forecast_report.head())


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/

        date forecasted_defects  number_of_defects
0 2023-12-21           Defect N           2.334319
1 2023-12-22           Defect N           2.324580
2 2023-12-23           Defect N           2.481991
3 2023-12-24           Defect N           2.520579
4 2023-12-25           Defect N           2.523765


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
